# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686588


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:37,  3.49s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:09,  2.56s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:48,  1.86s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:34,  1.38s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:13<00:00,  4.81it/s]

Rendering models:  90%|█████████ | 27/30 [00:13<00:00,  3.36it/s]

Rendering models:  93%|█████████▎| 28/30 [00:14<00:00,  3.28it/s]

Rendering models:  97%|█████████▋| 29/30 [00:14<00:00,  3.57it/s]

not-logged-in-3b85ef6ad3e2968c6631      0.000923
not-logged-in-7fffb28c67da974cf8f1      0.000595
Dynamatt                                0.000415
arjhun                                  0.008721
djswanso                                0.000437
rnevils                                 0.000388
jmfranci                                0.001633
CThomas                                 0.001267
tosnyder                                0.000517
ncaro                                   0.001035
not-logged-in-ea8ed61a7274cfb6a43b    112.179774
dejansson                               0.000447
Jessica_Korkmaz                         0.003725
not-logged-in-5f30b2bb9b7f9d49da41      3.266441
not-logged-in-5f30b2bb9b7f9d49da41      0.009679
gabrielweiss                            0.001194
clairedeu                               0.000461
pangeli5                                0.000501
dinosora                                0.000391
not-logged-in-a77c0eef6525bed230df      0.000501
Susymuller          

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())